In [1]:
#quickstart
from pathlib import Path
import pandas as pd
from bids import BIDSLayout
from joblib import Parallel, delayed
import nilearn
import nibabel as nib
import nltools.prefs as prf
from nilearn import datasets, image, plotting, input_data
from nilearn.masking import apply_mask
from nilearn.image import resample_to_img
from nilearn.image import concat_imgs, index_img
import numpy as np
import matplotlib.pylab as plt

import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')


import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from nltools.data import Brain_Data, Design_Matrix, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import zscore
from scipy.stats import binom, ttest_1samp
from sklearn.metrics import pairwise_distances
from nltools.stats import fdr, one_sample_permutation
from copy import deepcopy
from nltools.utils import get_anatomical
import nltools.plotting as nlplt
from datetime import date
from joblib import Parallel, delayed

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

from nilearn.input_data import NiftiMapsMasker,NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

import sys
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from statistics import mean
import matplotlib.style as style

#dat_paths = pd.read_csv('/data/MBDU/midla/notebooks/campcc/dat_paths_11272019.csv') 
#atlas = image.load_img('/data/MBDU/midla/notebooks/campcc/hox_msdl_atlas.nii.gz')
#sub1 = image.load_img('/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-20900/out/fmrip

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

/gpfs/gsfs11/users/MBDU/midla/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
data = pd.read_csv('/data/MBDU/midla/data/derivatives/fitlins/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bp-False_target-True_cosine-censor-trans-rot_test/ca_select.csv')

In [2]:
pd.read_csv('/data/MBDU/midla/data/derivatives/midla/primary_20200325/for_chris.csv')

,subject,session,task,run,total_n_session,scanner,acq_date,confounds_path,image_path,mask_path
0,21748,1,mid,2,4,3TC,2018-11-10,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
1,21748,2,mid,2,4,3TC,2019-03-23,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
2,21748,3,mid,2,4,3TC,2019-07-20,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
3,22228,1,mid,1,3,3TA,2019-04-20,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
4,22228,2,mid,1,3,3TA,2019-09-07,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
5,22228,3,mid,1,3,3TA,2020-01-04,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-03/func/sub-22228_ses-03_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-03/func/sub-22228_ses-03_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/se

In [20]:
data = data.loc[:,'contrast':'session']

In [23]:
data.columns

Index(['contrast', 'extension', 'path', 'run', 'session', 'space', 'stat', 'subject', 'suffix', 'task',
       ...
       'shaps_interval', 'mfq_interval', 'job_name', 'bp', 'target', 'cosine', 'censor', 'trans', 'rot', 'otherpsych'], dtype='object', length=228)

In [34]:
basc = datasets.fetch_atlas_basc_multiscale_2015(version='asym')
basc64 = basc['scale064']
atlas = image.load_img(basc64)
atlas_dat = atlas.get_fdata()

In [49]:
atlas_dat.shape

(53, 64, 52)

In [44]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

In [47]:
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,memory='nilearn_cache', verbose=5)

In [21]:
def get_roi_res(res, ses_dat, stat, roi, smooth=None):
    res = res.copy(deep=True)
    roi_mean = []
    for pp, row in res.iterrows():
        img = nli.load_img(row.path.as_posix())
        #img = nli.resample_to_img(img, roi_img)
        if smooth is not None:
            img = nli.smooth_img(img, smooth)
        dat = img.get_fdata().squeeze()
        roi_mean.append(dat[roi.astype(bool)].mean())
    res['vs_caudate'] = roi_mean
    res['session_number'] = res.session.astype(int)
    rr = res.merge(ses_dat, on=['subject', 'session_number'])
    return rr

In [52]:
pd.read_csv('/data/MBDU/midla/notebooks/campcc/merged_paths_for_icc.csv',index_col=0)

,subject,session,run,task_x,confounds_path_x,image_path_x,scaled_path_x,mask_path_x,ts_path,alt_ts_path,alt_rs_path,cfds_physio_path_x,confounds_path_y,image_path_y,mask_path_y,scaled_path_y,task_y,cfds_physio_path_y,task_ts_path,rest_ts_path
0,22409,1,1,mid,/data/MBDU/midla/data/derivatives/fitlins/nobutton_extras_nobsl_reml_2mm_strict/sub-22409/ses-01/run-1/out/out/fitlins/sub-22409/ses-01/sub-22409_ses-01_task-mid_run-1_design.tsv,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-22409/out/fmriprep/sub-22409/ses-01/func/sub-22409_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-22409/out/fmriprep/sub-22409/ses-01/func/sub-22409_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_desc-scale_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-22409/out/fmriprep/sub-22409/anat/sub-22409_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz,/data/MBDU/midla/notebooks/campcc/subject_ts/ts_subject_22409_session_1_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/alt_rs_conn/ts_subject_24204_session_1_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/alt_rs_conn/ts_subject_22409_session_1_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/physio/new_cfds_mid/new_cfds_mid_subject_22409_session_1_run_1_task_mid.tsv,/data/MBDU/midla/data/derivatives/fitlins/rest_dm_only/sub-22409/ses-01/run-1/out/out/fitlins/sub-22409/ses-01/sub-22409_ses-01_task-rest_run-1_design.tsv,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm_rest/sub-22409/out/fmriprep/sub-22409/ses-01/func/sub-22409_ses-01_task-rest_run-1_echo-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm_rest/sub-22409/out/fmriprep/sub-22409/anat/sub-22409_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm_rest/sub-22409/out/fmriprep/sub-22409/ses-01/func/sub-22409_ses-01_task-rest_run-1_echo-2_space-MNI152NLin2009cAsym_desc-scale_bold.nii.gz,rest,/data/MBDU/midla/notebooks/campcc/physio/new_cfds_rest/new_cfds_rest_subject_22409_session_1_run_1_task_rest.tsv,/data/MBDU/midla/notebooks/campcc/subject_ts/basc_ts_subject_22409_session_1_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/subject_ts/basc_ts_subject_22409_session_1_run_1_task_rest.csv
1,22409,2,1,mid,/data/MBDU/midla/data/derivatives/fitlins/nobutton_extras_nobsl_reml_2mm_strict/sub-22409/ses-02/run-1/out/out/fitlins/sub-22409/ses-02/sub-22409_ses-02_task-mid_run-1_design.tsv,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-22409/out/fmriprep/sub-22409/ses-02/func/sub-22409_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-22409/out/fmriprep/sub-22409/ses-02/func/sub-22409_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_desc-scale_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-22409/out/fmriprep/sub-22409/anat/sub-22409_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz,/data/MBDU/midla/notebooks/campcc/subject_ts/ts_subject_22409_session_2_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/alt_rs_conn/ts_subject_24204_session_1_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/alt_rs_conn/ts_subject_22409_session_2_run_1_task_mid.csv,/data/MBDU/midla/notebooks/campcc/physio/new_cfds_mid/new_cfds_mid_subject_22409_session_2_run_1_task_mid.tsv,/data/MBDU/midla/data/derivatives/fitlins/rest_dm_only/sub-22409/ses-02/run-1/out/out/fitlins/sub-22409/ses-02/sub-22409_ses-02_task-rest_run-1_design.tsv,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm_rest/sub-22409/out/fmriprep/sub-22409/ses-02/func/sub-22409_ses-02_task-rest_run-1_echo-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm_rest/sub-22409/out/fmriprep/sub-2

In [26]:
data = data.loc[(data.stat == 'effect'),:]

In [28]:
img = image.load_img(data.path[0])

In [51]:
img.get_data()[:,:,:,0].shape

(97, 115, 97)

In [33]:
img.get_data()[0,0,:]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],